In [1]:
from urllib.request import urlopen
from urllib import parse
from bs4 import BeautifulSoup
from Models.DataCollector import DataCollector
import torch.nn.functional as F
import json

In [2]:
class UserInspector(DataCollector):
    def __init__(self):
        self.name_to_key = {}
        self.name_to_id = {}
        self.id_to_key = {}
        self.key_to_id = {}
        with open('./metadata/name_to_key.json', 'r') as fp:
            self.name_to_key = json.load(fp)

        with open('./metadata/name_to_id.json', 'r') as fp:
            self.name_to_id = json.load(fp)

        with open('./metadata/id_to_key.json', 'r') as fp:
            self.id_to_key = json.load(fp)

        with open('./metadata/key_to_id.json', 'r') as fp:
            self.key_to_id = json.load(fp)

    def user_history_collector(self, userName):
        # url encoding (for Korean words)
        userName = parse.quote(userName)

        # open url and create bs4 Object
        html = urlopen("https://www.op.gg/summoner/userName="+userName)
        bsObject = BeautifulSoup(html, "html.parser")

        # decode userName from url encoding
        userName = parse.unquote(userName)

        # win/loss and play count
        wins = int(bsObject.find('span', 'wins').text[:-1])
        losses = int(bsObject.find('span', 'losses').text[:-1])
        total_play = wins + losses    
        win_rate = round(wins/total_play, 3)

        champ_list = bsObject.find_all("div", {'class' : 'ChampionBox Ranked'})
        champion_history = self.champ_history_setter(champ_list)            

        # this is (adapted) user_history DTO
        user_historyDTO = {
            'user_name' : userName,
            'total_play' : total_play,
            'win_rate' : win_rate,
            'champion_history' : champion_history,
        }
        return user_historyDTO

In [3]:
import torch

from Models.Models import AutoEncoder, Predictor, Classifier
from utils import champ_id_remap, global_win_rate, get_original_champ_id

In [4]:
class ChampionRecommender():
    def __init__(self):
        self.user_inspector = UserInspector()

        self.user_encoder = AutoEncoder(143, 12)
        self.user_encoder.load_state_dict(torch.load('./trained_model/user_encoder_augmented_new.pth'))

        self.item_encoder = AutoEncoder(143, 8)
        self.item_encoder.load_state_dict(torch.load('./trained_model/item_encoder2.pth'))

        self.predictor = Classifier(user_len=12, item_len=8, hidden_unit=15).eval()
        self.predictor.load_state_dict(torch.load('./trained_model/classifier_last_m.pth'))
        
        self.remapped_champ_id = champ_id_remap()
        self.global_win_rate = global_win_rate() 
        
        with open('./datasets/item_vectors_tf_idf.json', 'r') as fp:
            self.item_vectors = json.load(fp)        
        
        with open('./metadata/user_idf_table.json', 'r') as fp:
            self.idf_table = json.load(fp)

    def _recommender(self, userName):
        #try:
        user_data = self.user_inspector.user_history_collector(userName)
        win_rate_dict = self._winrate_predictior(user_data)
        return win_rate_dict
        #except:
        #    print("not a valid user. please check.")


    def _winrate_predictior(self, user_data):
        user_winrate = self._tensor_item(user_data['win_rate'])
        user_vec = self._user_vector_generator(user_data)
        played_champions = [i for i, e in enumerate(user_vec) if e != 0]
        user_vec = torch.Tensor(user_vec)
        user_vec = self.user_encoder.encoder(user_vec)
        
        win_rate_dict = dict()
        
        for key, item in self.item_vectors.items():
            if int(key) not in played_champions:
                global_win = self._tensor_item(self.global_win_rate[int(key)])
                item_vec = torch.Tensor([item])
                item_vec = self.item_encoder.encoder(item_vec)
                item_vec = item_vec.squeeze()
                
                user_vec = F.normalize(user_vec, dim=0)
                item_vec = F.normalize(item_vec, dim=0)
                
                prediction = self.predictor(user_vec, item_vec)
                prediction = torch.max(prediction, dim=0)
                win_rate_dict[key] = prediction[0]
        return win_rate_dict
    
    def _dict_origin_mapper(self, recommend_dict):
        original_recommend_dict = dict()
        for remapped_id, score in recommend_dict.items():
            original_id = get_original_champ_id(self.remapped_champ_id, int(remapped_id))
            original_name = self.user_inspector.key_to_id[str(original_id)]
            original_recommend_dict[original_name] = recommend_dict[remapped_id]
        return original_recommend_dict
            
            
    def _user_vector_generator(self, user_data):
        user_win_rate = user_data['win_rate']
        play_count_vector = [0 for x in range(143)]

        for champ_history in user_data['champion_history']:
            original_champ_id = champ_history['champion_key']
            champ_idx = self.remapped_champ_id[original_champ_id]

            max_play_count = self._get_max_play_count(user_data)
            measured_score = (champ_history['play_count'] / max_play_count) * self.idf_table[str(champ_idx)]
            play_count_vector[champ_idx] = measured_score
        
        return play_count_vector

    
    def _get_max_play_count(self, user_data):
        """
        Get count of maximally played champion for user_name
        """
        max_play_count = 0

        for champ_history in user_data['champion_history']:
            champ_play_count = champ_history['play_count']
            if max_play_count < champ_play_count:
                max_play_count = champ_play_count

        return max_play_count

    
    def _tensor_item(self, item):
        return torch.Tensor([item])

In [5]:
test = ChampionRecommender()

In [6]:
test_output = test._recommender('만년동핵폭탄')

In [7]:
result = test._dict_origin_mapper(test_output)

In [8]:
import operator
sorted_result = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [9]:
sorted_result

[('Kaisa', tensor(0.7433, grad_fn=<MaxBackward0>)),
 ('Ezreal', tensor(0.7375, grad_fn=<MaxBackward0>)),
 ('Veigar', tensor(0.7339, grad_fn=<MaxBackward0>)),
 ('Shaco', tensor(0.7325, grad_fn=<MaxBackward0>)),
 ('Zac', tensor(0.7295, grad_fn=<MaxBackward0>)),
 ('MissFortune', tensor(0.7249, grad_fn=<MaxBackward0>)),
 ('Jhin', tensor(0.7241, grad_fn=<MaxBackward0>)),
 ('Sivir', tensor(0.7086, grad_fn=<MaxBackward0>)),
 ('Karthus', tensor(0.7026, grad_fn=<MaxBackward0>)),
 ('Lucian', tensor(0.6992, grad_fn=<MaxBackward0>)),
 ('Kindred', tensor(0.6944, grad_fn=<MaxBackward0>)),
 ('Caitlyn', tensor(0.6941, grad_fn=<MaxBackward0>)),
 ('Vayne', tensor(0.6874, grad_fn=<MaxBackward0>)),
 ('Xayah', tensor(0.6735, grad_fn=<MaxBackward0>)),
 ('Fiora', tensor(0.6707, grad_fn=<MaxBackward0>)),
 ('Vladimir', tensor(0.6566, grad_fn=<MaxBackward0>)),
 ('Renekton', tensor(0.6472, grad_fn=<MaxBackward0>)),
 ('Aatrox', tensor(0.6444, grad_fn=<MaxBackward0>)),
 ('Syndra', tensor(0.6198, grad_fn=<MaxBackwa